# PCAD Notebook 1

This notebook works on a dataframe of metadata extracted from a file of MARC records by the `extract_pcad_from_marc.py` script. It clusters OCLC numbers and ISSNs and does a little data cleaning. During any PCAD run, you will run the extract script and this notebook twice: once for print records, and once for electronic records.

Required files/inputs: 
- `pcad_df_*{date}.pkl` file produced by `extract_pcad_from_marc.py`

Outputs:
- `.pkl` file of the dataframe after clustering and cleaning
- Optional outputs (when processing print data): two tab-delimited files, one of OCLC numbers and one of ISSNs, to create sets of electronic records in Alma

In [8]:
import pandas as pd
from tqdm import tqdm
import re
from datetime import date

Create a dataframe from the .pkl file produced by extract_pcad_from_marc.py.

In [9]:
#Change filename
pdf = pd.read_pickle('pcad_df_pserials_oct20_20201030.pkl')
pdf.shape
pdf

,MMS_ID,Title,OCN,ISSN,Related_OCNs,Related_ISSNs,Vol_nos,Gov_doc_nos
0,9920341180001701,Tutkimuksia Suomen maatalouden kannattavuudesta =,[9104278],[0438-9808],[],[],[],[]
0,9920358800001701,Pakistan,[25380502],[1061-6101],[],[],[],[]
0,9920370170001701,Abhandlungen der Königlich Preussischen Akade...,[10333878],[0233-2728],[],[],[],[]
0,9920380570001701,Zhongguo dian ying nian jian /,[9179105],[],[],[],[],[]
0,9920446280001701,Sports 'n spokes,[1114948794],[0161-6706],[],[],[],[]
...,...,...,...,...,...,...,...,...
0,9959871440001701,"Outlook (New York, N.Y. : 1893)",[5361126],[],[],[],[],[]
0,9959879870001701,Profile of health plans and utilization review...,[38314157],[],[],[],[],[]
0,9959887230001701,"Economic survey, Finland",[38364450],[1239-209X],[],"[1455-7606, 0532-9280]",[],[]
0,9959892490001701,Aakrosh :,[41503026],[0971-7862],[],[],[],[]


Combine resource ISSNs/OCNs with linked ISSNs/OCNs into a list for comparison

In [10]:
tqdm.pandas(desc="Progress: ")
pdf['OCN_cluster'] = pdf.progress_apply(lambda row: list(set(row['Related_OCNs'] + [x for x in row['OCN'] if x != ""])), axis=1)
pdf['ISSN_cluster'] = pdf.progress_apply(lambda row: list(set(row['Related_ISSNs'] + [x for x in row['ISSN'] if x != ""])), axis=1)
pdf['OCN_cluster'] = pdf.progress_apply(lambda row: list(set([x for x in row['OCN_cluster'] if x != ""])), axis=1)
pdf['ISSN_cluster'] = pdf.progress_apply(lambda row: list(set([x for x in row['ISSN_cluster'] if x != ""])), axis=1)
pdf

C:\Users\trail001\AppData\Local\Continuum\anaconda3\envs\isbnhld\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Progress: 100%|██████████████████████████████████████████████████████████| 103137/103137 [00:01<00:00, 67194.45it/s]


,MMS_ID,Title,OCN,ISSN,Related_OCNs,Related_ISSNs,Vol_nos,Gov_doc_nos,OCN_cluster,ISSN_cluster
0,9920341180001701,Tutkimuksia Suomen maatalouden kannattavuudesta =,[9104278],[0438-9808],[],[],[],[],[9104278],[0438-9808]
0,9920358800001701,Pakistan,[25380502],[1061-6101],[],[],[],[],[25380502],[1061-6101]
0,9920370170001701,Abhandlungen der Königlich Preussischen Akade...,[10333878],[0233-2728],[],[],[],[],[10333878],[0233-2728]
0,9920380570001701,Zhongguo dian ying nian jian /,[9179105],[],[],[],[],[],[9179105],[]
0,9920446280001701,Sports 'n spokes,[1114948794],[0161-6706],[],[],[],[],[1114948794],[0161-6706]
...,...,...,...,...,...,...,...,...,...,...
0,9959871440001701,"Outlook (New York, N.Y. : 1893)",[5361126],[],[],[],[],[],[5361126],[]
0,9959879870001701,Profile of health plans and utilization review...,[38314157],[],[],[],[],[],[38314157],[]
0,9959887230001701,"Economic survey, Finland",[38364450],[1239-209X],[],"[1455-7606, 0532-9280]",[],[],[38364450],"[0532-9280, 1455-7606, 1239-209X]"
0,9959892490001701,Aakrosh :,[41503026],[0971-7862],[],[],[],[],[41503026],[0971-7862]


Remove data that doesn't conform to the format we would expect for these identifiers

In [11]:
# remove any trailing periods
rmv_period = re.compile(r'^[0-9Xx]{4}\-[0-9Xx]{4}\.$')
# define the issn format as four dash four of numbers or the character X/x
regex = re.compile(r'^[0-9Xx]{4}\-[0-9Xx]{4}$')

In [12]:
pdf['ISSN_cluster'] = pdf.apply(lambda row: list(set([rmv_period.sub(issn[:-1], issn) for issn in row['ISSN_cluster']])), axis=1)
pdf['ISSN_cluster'] = pdf.apply(lambda row: list(filter(regex.search, row['ISSN_cluster'])), axis=1)
pdf

,MMS_ID,Title,OCN,ISSN,Related_OCNs,Related_ISSNs,Vol_nos,Gov_doc_nos,OCN_cluster,ISSN_cluster
0,9920341180001701,Tutkimuksia Suomen maatalouden kannattavuudesta =,[9104278],[0438-9808],[],[],[],[],[9104278],[0438-9808]
0,9920358800001701,Pakistan,[25380502],[1061-6101],[],[],[],[],[25380502],[1061-6101]
0,9920370170001701,Abhandlungen der Königlich Preussischen Akade...,[10333878],[0233-2728],[],[],[],[],[10333878],[0233-2728]
0,9920380570001701,Zhongguo dian ying nian jian /,[9179105],[],[],[],[],[],[9179105],[]
0,9920446280001701,Sports 'n spokes,[1114948794],[0161-6706],[],[],[],[],[1114948794],[0161-6706]
...,...,...,...,...,...,...,...,...,...,...
0,9959871440001701,"Outlook (New York, N.Y. : 1893)",[5361126],[],[],[],[],[],[5361126],[]
0,9959879870001701,Profile of health plans and utilization review...,[38314157],[],[],[],[],[],[38314157],[]
0,9959887230001701,"Economic survey, Finland",[38364450],[1239-209X],[],"[1455-7606, 0532-9280]",[],[],[38364450],"[0532-9280, 1455-7606, 1239-209X]"
0,9959892490001701,Aakrosh :,[41503026],[0971-7862],[],[],[],[],[41503026],[0971-7862]


In [13]:
lcno = re.compile(r'.*\(DLC\).*')
pdf['ISSN_cluster'] = pdf.apply(lambda row: list(set([i for i in row['ISSN_cluster'] if not lcno.search(i)])), axis=1)
pdf['OCN_cluster'] = pdf.apply(lambda row: list(set([i for i in row['OCN_cluster'] if not lcno.search(i)])), axis=1)
pdf['OCN_cluster'] = pdf.apply(lambda row: list(set([a.replace("ocm","").replace("OCM","").replace("ocn","").replace("OCN","").strip() for a in row['OCN_cluster']])), axis = 1)
pdf

,MMS_ID,Title,OCN,ISSN,Related_OCNs,Related_ISSNs,Vol_nos,Gov_doc_nos,OCN_cluster,ISSN_cluster
0,9920341180001701,Tutkimuksia Suomen maatalouden kannattavuudesta =,[9104278],[0438-9808],[],[],[],[],[9104278],[0438-9808]
0,9920358800001701,Pakistan,[25380502],[1061-6101],[],[],[],[],[25380502],[1061-6101]
0,9920370170001701,Abhandlungen der Königlich Preussischen Akade...,[10333878],[0233-2728],[],[],[],[],[10333878],[0233-2728]
0,9920380570001701,Zhongguo dian ying nian jian /,[9179105],[],[],[],[],[],[9179105],[]
0,9920446280001701,Sports 'n spokes,[1114948794],[0161-6706],[],[],[],[],[1114948794],[0161-6706]
...,...,...,...,...,...,...,...,...,...,...
0,9959871440001701,"Outlook (New York, N.Y. : 1893)",[5361126],[],[],[],[],[],[5361126],[]
0,9959879870001701,Profile of health plans and utilization review...,[38314157],[],[],[],[],[],[38314157],[]
0,9959887230001701,"Economic survey, Finland",[38364450],[1239-209X],[],"[1455-7606, 0532-9280]",[],[],[38364450],"[0532-9280, 1455-7606, 1239-209X]"
0,9959892490001701,Aakrosh :,[41503026],[0971-7862],[],[],[],[],[41503026],[0971-7862]


In [14]:
#Change filename
today = str(date.today()).replace('-','')
pdf.to_pickle(f'cleaned_print_{today}.pkl')

Get lists of OCNs and ISSNs that we can search for in electronic resource records in Alma to try to find matches based on identifiers. Skip if no need to produce ID lists for Alma.

In [ ]:
pdf = pd.read_pickle('cleaned_print_20201106.pkl')
pdf

In [ ]:
OCNs = []
for x in pdf['OCN_cluster']:
    for y in x:
        OCNs.append(f'(OCoLC){y}')
OCNs

In [ ]:
ISSNs = []
for x in pdf['ISSN_cluster']:
    for y in x:
        ISSNs.append(y)
ISSNs

In [ ]:
#dedup OCNs
print(len(OCNs))
OCNs = list(set(OCNs))
print(len(OCNs))

In [ ]:
#dedup ISSNs
print(len(ISSNs))
ISSNs = list(set(ISSNs))
print(len(ISSNs))

In [ ]:
#change filenames if desired
odf = pd.DataFrame(OCNs, columns=['OCLC number'])
odf = odf.rename(columns={'OCLC number': '035 field'})
odf.to_csv(f'ocns-extracted-from-print-recs_{today}.csv', sep='\t',index=False)
idf = pd.DataFrame(ISSNs, columns=['ISSN'])
idf.to_csv(f'issns-extracted-from-print-recs_{today}.csv', sep='\t',index=False)